In [1]:
import pandas as pd
from datetime import datetime
import json

In [2]:
clients = {}
df = pd.read_csv(
    "metricscontribution_rate_initiative_cross_client_data_1613_for_students.csv")
client_ids = df.client_id.unique()
for client_id in client_ids:
    client_id = str(client_id)
    clients.update(dict({client_id: {}}))

# print(clients)

In [24]:
for client_id in client_ids:
    client_specific_df = df[df["client_id"] == client_id][["period_end", "source", "source_type", "attribution", "customers"]]

    # Convert "period_end" to datetime with UTC and sort the DataFrame
    client_specific_df["period_end"] = pd.to_datetime(client_specific_df["period_end"], format="ISO8601", utc=True)
    client_specific_df = client_specific_df.sort_values(by="period_end")

    # Group by the period (e.g., week or month)
    grouped = client_specific_df.groupby(client_specific_df["period_end"].dt.to_period("W"))

    # Update the dictionary for the client with periods
    period_dict = {}
    for period, period_df in grouped:
        period_data = {}

        # Find the number of attributed and unattributed customers
        attributed_customers = period_df[period_df['attribution'] == 'ATTRIBUTED']['customers'].sum()
        unattributed_customers = period_df[period_df['attribution'] != 'ATTRIBUTED']['customers'].sum()

        # Update the period data with the count of ATTRIBUTED and UNATTRIBUTED count
        # and the contribution rate for this period
        period_data["period"] = period.start_time.isoformat()  # Save the date in ISO8601 format
        period_data["ATTRIBUTED"] = int(attributed_customers)
        period_data["UNATTRIBUTED"] = int(unattributed_customers)

        # Calculate "CR" (Contribution Rate) safely
        if attributed_customers + unattributed_customers == 0:
            period_data["CR"] = 0
        else:
            period_data["CR"] = (attributed_customers / (attributed_customers + unattributed_customers) * 100)

        # Find all the records for the period
        records = []
        sources = period_df['source'].unique()
        for source in sources:
            source_df = period_df[period_df['source'] == source]
            source_data = {
                'source': str(source),
                'source_type': str(source_df.iloc[0]['source_type']),
                'ATTRIBUTED': str(source_df['attribution'].eq('ATTRIBUTED').any()),
                'customers': int(source_df['customers'].sum())
            }
            records.append(source_data)

        period_data["records"] = records
        # Append the period data to the list for this client
        period_dict[period.start_time.strftime('%Y-%m-%dT%H:%M:%S%z')] = period_data

    # Add everything to the final dictionary for this client
    clients[str(client_id)] = period_dict

/tmp/ipykernel_10215/28526579.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = client_specific_df.groupby(client_specific_df["period_end"].dt.to_period("W"))
/tmp/ipykernel_10215/28526579.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = client_specific_df.groupby(client_specific_df["period_end"].dt.to_period("W"))
/tmp/ipykernel_10215/28526579.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = client_specific_df.groupby(client_specific_df["period_end"].dt.to_period("W"))
/tmp/ipykernel_10215/28526579.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = client_specific_df.groupby(client_specific_df["period_end"].dt.to_period("W"))
/tmp/ipykernel_10215/28526579.py:9: UserWarning: Converting to PeriodArray/Index representation will drop timezone informati

In [25]:
print(clients["2074305340"])
file_path = "clients.json"
json_string = json.dumps(clients)
with open(file_path, 'w') as f:
    f.write(json_string)

{'2023-01-02T00:00:00': {'period': '2023-01-02T00:00:00', 'ATTRIBUTED': 1, 'UNATTRIBUTED': 47, 'CR': 2.083333333333333, 'records': [{'source': 'global_footer', 'source_type': 'zone', 'ATTRIBUTED': 'True', 'customers': 1}, {'source': 'direct', 'source_type': 'link', 'ATTRIBUTED': 'False', 'customers': 47}]}, '2023-01-09T00:00:00': {'period': '2023-01-09T00:00:00', 'ATTRIBUTED': 1, 'UNATTRIBUTED': 52, 'CR': 1.8867924528301887, 'records': [{'source': 'direct', 'source_type': 'link', 'ATTRIBUTED': 'False', 'customers': 52}, {'source': 'global_footer', 'source_type': 'zone', 'ATTRIBUTED': 'True', 'customers': 1}]}, '2023-01-16T00:00:00': {'period': '2023-01-16T00:00:00', 'ATTRIBUTED': 1, 'UNATTRIBUTED': 59, 'CR': 1.6666666666666667, 'records': [{'source': 'direct', 'source_type': 'link', 'ATTRIBUTED': 'False', 'customers': 59}, {'source': 'advocate_welcome_email', 'source_type': 'link', 'ATTRIBUTED': 'True', 'customers': 1}]}, '2023-01-23T00:00:00': {'period': '2023-01-23T00:00:00', 'ATTRIB